In [1]:
import pandas as pd
import os
sources_list = [file for file in os.listdir('../Sources/3.-Population/') if file.endswith('.csv')]
print(sources_list)
for file in sources_list:
    globals()[file[:-4]] = pd.read_csv('../Sources/3.-Population/' + file)
external = [file for file in os.listdir('../Sources/External_sources/') if file.endswith('.csv')]
print(external)
for file in external:
    globals()[file[:-4]] = pd.read_csv('../Sources/External_sources/' + file)

['immigrants_emigrants_by_sex.csv', 'population.csv', 'deaths.csv', 'immigrants_emigrants_by_destination.csv', 'most_frequent_baby_names.csv', 'immigrants_by_nationality.csv', 'births.csv', 'most_frequent_names.csv', 'immigrants_emigrants_by_age.csv']
['_est_salut_publica_esp_vida.csv', '_distribucio_territorial_renda_familiar.csv']


In [2]:
#We eliminate all variables for years !=2017
births_2017 = births[births["Year"] == 2017]
births_2017.columns = ['Year', 'District_Code', 'District_Name', 'Neighborhood_Code',
       'Neighborhood_Name', 'Gender', 'Births']



In [3]:
renta = _distribucio_territorial_renda_familiar

In [4]:
renta.rename(columns = {"Índex RFD Barcelona = 100":"Revenue_Index"},inplace = True)
#We create 4 revenue categories for the negihborhoods based on their revenue levels
renta["Category"] = ["Very poor" if e < 63 
                                          else "Poor" if e < 79
                                          else "Low middle class" if e < 100
                                          else "High middle class" if e < 126
                                          else "Rich" if e < 159
                                          else "Very rich"
                                          for e in renta["Revenue_Index"]]
renta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Any             73 non-null     int64  
 1   Codi_Districte  73 non-null     int64  
 2   Nom_Districte   73 non-null     object 
 3   Codi_Barri      73 non-null     int64  
 4   Nom_Barri       73 non-null     object 
 5   Població        73 non-null     int64  
 6   Revenue_Index   73 non-null     float64
 7   Category        73 non-null     object 
dtypes: float64(1), int64(4), object(3)
memory usage: 4.7+ KB


In [5]:
renta.columns = ['Any', 'District_Code', 'District_Name', 'Neighborhood_Code', 'Neighborhood_Name',
       'Population', 'Revenue_Index', 'Category']
renta

,Any,District_Code,District_Name,Neighborhood_Code,Neighborhood_Name,Population,Revenue_Index,Category
0,2017,1,Ciutat Vella,1,el Raval,47986,71.2,Poor
1,2017,1,Ciutat Vella,2,el Barri Gòtic,16240,106.1,High middle class
2,2017,1,Ciutat Vella,3,la Barceloneta,15101,79.6,Low middle class
3,2017,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",22923,99.4,Low middle class
4,2017,2,Eixample,5,el Fort Pienc,32048,106.5,High middle class
...,...,...,...,...,...,...,...,...
68,2017,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,13710,150.1,Rich
69,2017,10,Sant Martí,70,el Besòs i el Maresme,22893,60.4,Very poor
70,2017,10,Sant Martí,71,Provençals del Poblenou,20649,102.3,High middle class
71,2017,10,Sant Martí,72,Sant Martí de Provençals,26187,67.4,Poor


In [6]:
renta["Population"].sum()
#Doubles the population when merging
#Population of Barcelona in 2017 was 5,4M vs 1,6M in data

1625137

In [9]:
#remove columns that we don't need
birth_rent = pd.merge(births_2017,renta,how="left",on='Neighborhood_Code')
birth_rent.drop("Any",axis=1,inplace=True)
birth_rent.drop("District_Code_y",axis=1,inplace=True)
birth_rent.drop("District_Name_y",axis=1,inplace=True)
birth_rent.drop("Neighborhood_Name_y",axis=1,inplace=True)

In [10]:
birth_rent.columns = ['Year', 'District_Code', 'District_Name', 'Neighborhood_Code',
       'Neighborhood_Name', 'Gender', 'Births', 'Population',
       'Revenue_Index', 'Category']

In [11]:
birth_rent

,Year,District_Code,District_Name,Neighborhood_Code,Neighborhood_Name,Gender,Births,Population,Revenue_Index,Category
0,2017,1,Ciutat Vella,1,el Raval,Boys,283,47986,71.2,Poor
1,2017,1,Ciutat Vella,2,el Barri Gòtic,Boys,56,16240,106.1,High middle class
2,2017,1,Ciutat Vella,3,la Barceloneta,Boys,51,15101,79.6,Low middle class
3,2017,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Boys,90,22923,99.4,Low middle class
4,2017,2,Eixample,5,el Fort Pienc,Boys,117,32048,106.5,High middle class
...,...,...,...,...,...,...,...,...,...,...
141,2017,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,Girls,58,13710,150.1,Rich
142,2017,10,Sant Martí,70,el Besòs i el Maresme,Girls,99,22893,60.4,Very poor
143,2017,10,Sant Martí,71,Provençals del Poblenou,Girls,80,20649,102.3,High middle class
144,2017,10,Sant Martí,72,Sant Martí de Provençals,Girls,98,26187,67.4,Poor


In [13]:
total_birth = birth_rent.groupby("Category", as_index=False)[["Births"]].sum()
total_birth

,Category,Births
0,High middle class,3417
1,Low middle class,3562
2,Poor,2409
3,Rich,803
4,Very poor,1893
5,Very rich,1442


In [19]:
birth_g = birth_rent.groupby(["Category","Gender"],as_index=False)[["Births"]].sum()

In [22]:
#birth_gender = birth_rent.groupby(["Category","Gender"],as_index=False)[["Births"]].sum()
#birth_gender["Gender_Birth_%"] = birth_gender["Births"] /  total_birth["Births"] 
birth_gender1 = pd.merge(birth_g,total_birth,how="inner",on="Category")
birth_gender1.columns = ['Category', 'Gender', 'Births', 'Total_Births']
birth_gender = birth_gender1.groupby(['Category','Gender']).first()
birth_gender["Gender_Birth_%"] = birth_gender["Births"] / birth_gender["Total_Births"]
birth_gender["Gender_Birth_%"] = pd.Series(["{0:.2f}%".format(val * 100) for val in birth_gender["Gender_Birth_%"]], index = birth_gender.index)
birth_gender.drop("Total_Births",axis=1,inplace=True)
birth_gender = birth_gender.unstack()
birth_gender

Births       Gender_Birth_%        
Gender              Boys Girls           Boys   Girls
Category                                             
High middle class   1773  1644         51.89%  48.11%
Low middle class    1803  1759         50.62%  49.38%
Poor                1243  1166         51.60%  48.40%
Rich                 423   380         52.68%  47.32%
Very poor            979   914         51.72%  48.28%
Very rich            757   685         52.50%  47.50%

In [25]:
birth_rentp = birth_rent.groupby(["Category"],as_index=False)[["Births","Population"]].sum()
birth_rentp["Population"] = (birth_rentp["Population"] /2).astype(int)
#We have to divide population by 2, because when merging it got duplicated 
birth_rentp["Birth rate"] = birth_rentp["Births"] / birth_rentp["Population"]
birth_rentp['Birth rate'] = pd.Series(["{0:.2f}%".format(val * 100) for val in birth_rentp['Birth rate']], index = birth_rentp.index)
birth_rentp.sort_values(by = "Birth rate",ascending=False,inplace = True)
birth_rentp

,Category,Births,Population,Birth rate
4,Very poor,1893,210499,0.90%
3,Rich,803,90528,0.89%
2,Poor,2409,282609,0.85%
0,High middle class,3417,416741,0.82%
1,Low middle class,3562,443565,0.80%
5,Very rich,1442,181195,0.80%


In [27]:
with pd.ExcelWriter("../Results.xlsx", engine="openpyxl", mode="a") as writer: 
    total_birth.to_excel(writer, sheet_name='Total births')
    birth_gender.to_excel(writer, sheet_name='Births by gender')
    birth_rentp.to_excel(writer, sheet_name='Birth rates by cat')